# Deploying `langgraph_bedrock.py` to AWS

This notebook demonstrates how to deploy `langgraph_bedrock.py` to AWS. 

It will build the Docker image and deploy it to your AWS environment.

For more samples and advanced capabilities, check out the AWS-provided repository:  
[Amazon Bedrock AgentCore Samples](https://github.com/awslabs/amazon-bedrock-agentcore-samples/)


In [ ]:
import os
from bedrock_agentcore_starter_toolkit import Runtime

# 1. Clear any old AWS credentials. There are times I'm having access issues due to expired creds, so by default i clear it as a best practice.
os.environ.pop("AWS_ACCESS_KEY_ID", None)
os.environ.pop("AWS_SECRET_ACCESS_KEY", None)
os.environ.pop("AWS_SESSION_TOKEN", None)

# 2. Set your SSO profile, here I'm utilizing aws sso login
os.environ["AWS_PROFILE"] = "sandbox"  


# 4. Initialize Runtime
agentcore_runtime = Runtime()

# 5. Configure the agent
agent_name = "ticker_checker_agent"
response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py", 
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region="us-east-1",
    agent_name=agent_name
)

print("AgentCore configuration response:")
print(response)

### Deploy the configuration

In [ ]:
launch_result = agentcore_runtime.launch()


### Check for the status of the deployment

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

In [ ]:
from IPython.display import Markdown, display
import json

invoke_response = agentcore_runtime.invoke({"prompt": "Whats the btc price today"})
response_text = invoke_response['response'][0]
display(Markdown(response_text))